In [1]:
# import necessary libraries
import numpy as np

# Load PGL libraries and start a PGL window
from pgl import pgl, pglTask, pglExperiment, pglParameter, pglParameterBlock
pgl = pgl()

# close any existing windows
pgl.cleanUp()

================================ pglBase: init =================================
(pgl) mglMetal error log can be viewed in MacOS Console app by searching for PROCESS mglMetal or in a terminal with:
      log stream --level info --process mglMetal
(pgl) To search for something specifc, e.g. messages from mglMovie:
      log stream --predicate 'eventMessage CONTAINS "mglMovie"' --style syslog --level info
(pgl:checkOS) Python version: 3.12.3 | packaged by conda-forge | (main, Apr 15 2024, 18:35:20) [Clang 16.0.6 ]
(pgl:checkOS) Running on MacBook Pro (MacBookPro18,3) with macOS version: 26.2
(pgl:checkOS) Apple M1 Pro Cores: 8 (6 performance and 2 efficiency) Memory: 32 GB
(pgl:checkOS) GPU: Apple M1 Pro (Built-In) 14 cores, Metal 4 support
(pgl:checkOS)   Color LCD [Main Display]: 3024 x 1964 Retina (Built-in Liquid Retina XDR Display) GammaTable size: 1024
(pglBase) Main library instance created
(pglBase:removeOrphanedSockets) No orphaned sockets found in /Users/justin/Library/Containe

In [ ]:
# Set up task class
class pglRandomDotTask(pglTask):
    
    ########################
    def __init__(self, pgl):
        super().__init__()
        
        # set task parameters, these will automatically be saved in the settings file
        self.settings.taskName = "Random Dot Motion Task"
        self.settings.seglen = [1, 0.5]
        
        # fixed parameters, these will automatically be saved in the settings file
        self.settings.fixedParameters = {
            'width':15,
            'height':10,
            'coherence':(0.1,1),
            'dir':np.arange(0,360,45)
        }        
        p = self.settings.fixedParameters

        # add parameters for coherence and direction
        coherence = pglParameter('coherence',p['coherence'])
        dir = pglParameter('dir',p['dir'])
        self.addParameter(pglParameterBlock([dir, coherence]))
        
        # initalize stimulus
        self.rdk = pgl.randomDots(width=p['width'], height=p['height'])
    ########################
    def updateScreen(self):
        if self.currentSegment==0:
            self.rdk.display(direction=self.currentParams['dir'], coherence=self.currentParams['coherence'], speed=5.0)

# initialize task
randomDotTask = pglRandomDotTask(pgl)



In [ ]:
# Set up experiment
e = pglExperiment(pgl, "window")
e.initScreen()

# add the random dot motion task
e.addTask(randomDotTask)

# and run the experiment
e.run()

In [ ]:
from pgl import pglStaircaseUpDown
# Set up task class
class pglFixationTaskLeftRight(pglTask):
    
    ########################
    def __init__(self, pgl):
        super().__init__()
        # task name
        self.settings.taskName = "Fixation Task Left Right"
        self.settings.fixedParameters = {
            'fixSize':1
        }
        self.fixSize = self.settings.fixedParameters['fixSize']
        
        # segments. Fixation, Stimulus, Response
        self.settings.seglen = [0.2, 0.5, 3.0]
        
        # add a parameter for which side of the fixation cross dims
        self.addParameter(pglParameter('side',(-1,1)))
        
        # initialize stairase
        self.staircase = pglStaircaseUpDown()
        self.staircase.startStaircase()
    
    ########################
    def startSegment(self, startTime):
        '''
        Start a segment.
        '''
        super().startSegment(startTime)
        # display stimulus only during first segment
        if self.currentSegment==0:
            self.luminance = (1-self.staircase.get())
            print(f"  Luminance: {self.luminance:.2f}")

    ########################
    def updateScreen(self):
        
        # default left and right colors
        leftColor = rightColor = 1.0
        
        if self.currentSegment==1:
            # dim left or right side of fixation cross
            if self.currentParams['side']==-1:
                leftColor = self.luminance
            else:
                rightColor = self.luminance
            #leftColor = 0.0
            #rightColor = 1.0

        # draw the left side
        self.pgl.line(-self.fixSize/2, 0, 0, 0, leftColor)
        # draw the right side
        self.pgl.line(0, 0, self.fixSize/2, 0, rightColor)

        # draw the vertical line
        self.pgl.line(0, -self.fixSize/2, 0, self.fixSize/2, 1.0)            

# initialize task
fixationTask = pglFixationTaskLeftRight(pgl)

In [3]:
# Set up experiment
e = pglExperiment(pgl, "window")
e.initScreen()

# add the random dot motion task
e.addTask(fixationTask)

# and run the experiment
e.run()

(pglSettingsManager:loadSettings) Loading settings from '/Users/justin/.pgl/settings/window.json'.
(pglSettings) 'startOnVolumeTrigger' not found in JSON file /Users/justin/.pgl/settings/window.json, using default False
================================= pglBase:open =================================
(pglBase:open) Starting mglMetal application: /Users/justin/Library/Developer/Xcode/DerivedData/Build/Products/Debug/mglMetal.app
(pglBase:open) Using socket with address: /Users/justin/Library/Containers/gru.mglMetal/Data/pglMetal.socket.20260220_232746.MISrHfIlyK
(pgl:_pglComm) .Connected to: /Users/justin/Library/Containers/gru.mglMetal/Data/pglMetal.socket.20260220_232746.MISrHfIlyK
(pgl:_resolution:getResolution) Display 0/1: 1512x982 120Hz 32bits
(pglKeyboardMouse:start) Starting keyboard and mouse event listener.
(pglDevices) Added device: pglKeyboard
(pglEventListener) Eating 7 keys: [18, 19, 20, 21, 49, 50, 53]
(pglImage:imageDelete) Deleting image 1 (653x51)
(pglExperiment:startPh